# Simple models

This notebook picks selected simple models from the [LeNet page](http://yann.lecun.com/exdb/mnist/) and performs setup as described in `[1]`, evaluating their performance on the MNIST classification task.  The evaluation uses 5-fold cross-validation with a stratified split strategy, and reports the error rate.

`[1]` - [Gradient-Based Learning Applied to Document Recognition](http://yann.lecun.com/exdb/publis/pdf/lecun-98.pdf),  LeCun et al, Nov 1998

In [2]:
# Load in dependencies, may not use all of these
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import pickle

%matplotlib inline

## Load pickle files

In [3]:
# Set up the file directory and names
DIR = '../input/'
X_TRAIN = DIR + 'train-images-idx3-ubyte.pkl'
Y_TRAIN = DIR + 'train-labels-idx1-ubyte.pkl'
X_TEST = DIR + 't10k-images-idx3-ubyte.pkl'
Y_TEST = DIR + 't10k-labels-idx1-ubyte.pkl'

print('Loading pickle files ...')
X_train = pickle.load( open( X_TRAIN, "rb" ) )
y_train = pickle.load( open( Y_TRAIN, "rb" ) )
X_test = pickle.load( open( X_TEST, "rb" ) )
y_test = pickle.load( open( Y_TEST, "rb" ) )

n_train = X_train.shape[0]
n_test = X_test.shape[0]

w = X_train.shape[1]
h = X_train.shape[2]

# Reshape the images so they're a single row in the numpy array
X_train = X_train.reshape((n_train, w * h))
X_test = X_test.reshape((n_test, w * h))
y_train = y_train.squeeze()
y_test = y_test.squeeze()

assert X_train.shape[0] == y_train.shape[0]
assert X_test.shape[0] == y_test.shape[0]

print('Train images shape {}, labels shape {}'.format(X_train.shape, y_train.shape))
print('Test images shape {}, labels shape {}'.format(X_test.shape, y_test.shape))

Loading pickle files ...
Train images shape (60000, 784), labels shape (60000,)
Test images shape (10000, 784), labels shape (10000,)


## Helper functions

Before trying a few different algorithms out, let's define a reusable set of functions to cross-validate and predict on the test set. Because scikit-learn has such a uniform interface, we can re-use these on pretty much any classification algorithm out there.

In [4]:
from sklearn.cross_validation import StratifiedKFold, cross_val_score

SEED = 1234 # Fix the seed for repeatability

def evaluate_model(model, scoring, X, y, n_examples, n_folds, n_jobs, random_state):
    ''' Evaluates the performance of a model using X and y with n_folds cross validation'''
    if n_examples == -1:
        count = 'all ' + str(X.shape[0])
    else:
        count = str(n_examples)
    print('Running {}-fold cross validation on {} examples ..'.format(n_folds, count))
    X = X[:n_examples,]
    y = y[:n_examples,]
    cv = StratifiedKFold(y, n_folds, shuffle=True, random_state=random_state)
    accuracies = cross_val_score(estimator=model, X=X, y=y, cv=cv, scoring=scoring, n_jobs=n_jobs
                                , verbose=0)
    errors = [1 - acc for acc in accuracies]
    return errors

def train_model(model, X, y, n_examples, random_state):
    ''' Trains and returns model'''
    print('Training model with {} examples ..'.format(n_examples))
    model_fit = model.fit(X[:n_examples,], y[:n_examples,])
    return model_fit

def train_and_predict(model, X_train, y_train, X_test, n_examples, random_state):
    ''' Trains and makes prediction'''
    fit_model = train_model(model, X_train[:n_examples], y_train[:n_examples], n_examples, random_state)
    y_test_pred = fit_model.predict(X_test) # Don't restrict size of X_test to n_examples
    return y_test_pred

def train_and_evaluate(model, X_train, y_train, X_test, y_test, n_examples, random_state):
    ''' Trains the model, predicts on test set, evaluates performance'''
    fit_model = train_model(model, X_train[:n_examples], y_train[:n_examples], n_examples, random_state)
    acc = fit_model.score(X_test, y_test)
    error = 1.0 - acc
    return error
    
def mean_std(values):
    '''Calculates the mean and std dev of the input values
    INPUT: List or numpy array of values
    RETURNS (mean, std)
    '''
    mean = np.mean(values)
    std = np.std(values)
    return mean, std

/Users/tim/anaconda/envs/tensorflow/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## [1] C.1 - Logistic regression

Logistic regression is a simple model to train that's a good baseline to work from. We're using the `multinomial` option for the model, which minimizes the error across the entire probability distribution. This most closely corresponds to the 'Pairwise linear' error rate of 7.6% in `[1]`.  Our model has a 5-fold CV error of 8.1%  on the training set, and test error of 7.49%. This is 0.11% different to the 7.6% error reported in [1] C.1, pretty close.

In [7]:
from sklearn.linear_model import LogisticRegression

NUM_EXAMPLES = -1 # -1 uses all examples

logreg_model = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=10000, verbose=3, random_state=SEED) 
errors = evaluate_model(logreg_model, 'accuracy', X_train, y_train, NUM_EXAMPLES, 
                        n_folds=5, n_jobs=-2, random_state=SEED)
cv_error_mean, cv_error_std = mean_std(errors)

test_error = train_and_evaluate(logreg_model, X_train, y_train, X_test, y_test, NUM_EXAMPLES, SEED)

print('Logistic regression CV on {} examples. Mean error {}, Std dev {}'.format(NUM_EXAMPLES, cv_error_mean, cv_error_std))
print('Logistic regression error {}'.format(test_error))

Running 5-fold cross validation on all 60000 examples ..


KeyboardInterrupt: 

## [1] C.2 k-Nearest Neighbours

Let's try kNN to classify the numbers.  This is a non-linear classification algorithm that accepts a single hyperparameter to control the number of neighbours, `k`.  [1] C.2. used  a `k` of 3, which gave an error rate of 5% on the test set.  Let's try a few different values of  `k` just to see how the performance varies. 

In [10]:
from sklearn.neighbors import KNeighborsClassifier

NUM_EXAMPLES = -1 # -1 uses all examples

knn_model = KNeighborsClassifier(n_neighbors=3, n_jobs=-2)
# errors = evaluate_model(knn_model, 'accuracy', X_train, y_train, NUM_EXAMPLES, 
#                         n_folds=5, n_jobs=-2, random_state=SEED)
# cv_error_mean, cv_error_std = mean_std(errors)

test_error = train_and_evaluate(knn_model, X_train, y_train, X_test, y_test, NUM_EXAMPLES, SEED)

print('kNN CV on {} examples. Mean error {}, Std dev {}'.format(NUM_EXAMPLES, cv_error_mean, cv_error_std))
print('kNN test error {}'.format(test_error))

Training model with -1 examples ..
kNN CV on -1 examples. Mean error 0.08101812864776892, Std dev 0.0019186396368524705
kNN test error 0.02959999999999996


In [ ]:
from sklearn.model_selection import GridSearchCV

NUM_EXAMPLES = -1 # -1 uses all examples


# Finding best value of `k` on training data
fit_params = {'n_neighbors' : [1, 2, 3, 4, 5]}
knn_cv = StratifiedKFold(y_train, n_folds=5, shuffle=True, random_state=SEED)

knn_model = KNeighborsClassifier()
grid_knn = GridSearchCV(knn_model, fit_params, n_jobs=-1)
grid_knn.fit(X_train[:NUM_EXAMPLES], y_train[:NUM_EXAMPLES])
pd.DataFrame(grid_knn.cv_results_)



The class distribution is pretty evenly split between the classes. 1 is the most popular class with 11.24% of instances, and at the other end 5 is the least frequent class, with 9.04% of instances

In [ ]:
# Test label distribution
y_test_df = pd.DataFrame(y_test, columns=['class'])
y_test_df.plot.hist(legend=False, bins=10)
test_counts = y_test_df['class'].value_counts(normalize=True)
hist_df['test'] = test_counts

The distribution looks very similar between training and test datasets.

In [ ]:
hist_df['diff'] = np.abs(hist_df['train'] - hist_df['test'])
hist_df.sort_values('diff', ascending=False)['diff'].plot.bar()

The largest difference is 0.0040% in the number 2 class.

In [ ]:
# Final 
assert X_train.dtype == np.uint8
assert y_train.dtype == np.uint8
assert X_test.dtype == np.uint8
assert y_test.dtype == np.uint8